In [5]:
import sys
sys.path.append('./utils_library/')
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import cv2
import visualization.visualizations as vis
from applications.model_wrapper import ModelWrapper
import post_processing.post as post
import configs.draw_config as draw_config
import time


def preprocess_img(img):
    """
    This function implements the preprocessing of input image.
    Input original img, it will:
    1. add newaxis to first dimension
    2. fix image height to 368 and proportionally resize image width
    3. normalize to 0...1 by dividing 255
    """
    inHeight = img.shape[0]
    inWidth = img.shape[1]
    frameHeight = 368
    frameWidth = round(frameHeight/inHeight * inWidth)

    input_img = tf.image.resize(img, (frameHeight, frameWidth))
    input_img = tf.image.convert_image_dtype(input_img, dtype=tf.float32)

    input_img /= 255
    input_img = input_img[tf.newaxis, ...]

    return input_img


def process_image(img_path):
    # load images and do preprocessing
    img_bgr = cv2.imread(img_path)
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    input_img = preprocess_img(img_rgb)

    # predict and post-processing
    start_time = time.time()
    pafs, kpts = model.predict(input_img)
    end_time = time.time()
    model_time = end_time - start_time

    start_time = time.time()
    skeletonizer = post.Skeletonizer(kpts[0], pafs[0])
    skeletons = skeletonizer.create_skeletons()
    end_time = time.time()
    post_time = end_time - start_time

    skeleton_drawer = vis.SkeletonDrawer(img_rgb, draw_config)
    for skeleton in skeletons:
        skeleton.draw_skeleton(skeleton_drawer.joint_draw, skeleton_drawer.kpt_draw)

    return skeletons, img_rgb, model_time, post_time

In [7]:
## configs
#model_path = "./trained_models/model11_test-15Sun1219-2101/"
model_name = "mobilenet_model_COCO+RGB_extra"
model_path = "./trained_models/" + model_name
img_dir = "./self_dataset/RGB_Dataset/RGB_imgs/"

# loading model and check architecture
model = tf.keras.models.load_model(model_path)

In [8]:
# creating json file
RESULT_ROOT = './results_full/'
result_dir = RESULT_ROOT + model_name + '/NMS_v2/'
#result_dir = RESULT_ROOT + 'orignal_model/NMS_v2/'
result_json_file = result_dir + 'record.json'
result_imgs = result_dir + 'images/'

import os
if not os.path.isdir(result_dir):
    os.makedirs(result_dir, )
if not os.path.isdir(result_imgs):
    os.mkdir(result_imgs)

import json
result_dict = {}
img_num = 0
model_t_sum = 0
post_t_sum = 0

for line in open("./self_dataset/imgL_val.txt"):
    img_num += 1
    img_name = line[-41:-1]
    print(img_num, "/65 :", img_name)
    img_path = img_dir + img_name
    
    skeletons, img_rgb, model_time, post_time = process_image(img_path)
    
    model_t_sum += model_time
    post_t_sum += post_time
    
    temp = {}
    n = 1
    for skeleton in skeletons:
        person_name = "person" + str(n)
        temp[person_name] = {}
        for name in skeleton.keypoints:
            temp[person_name][name] = [skeleton.keypoints[name][0] * img_rgb.shape[0],
                                              skeleton.keypoints[name][1] * img_rgb.shape[1]]
        n += 1
    
    result_dict[img_name] = temp
    plt.imsave(result_imgs+img_name, img_rgb)
    print("Average model time usage: ", model_t_sum/img_num)
    print("Average post time usage: ", post_t_sum/img_num)

result_json = json.dumps(result_dict)
with open(result_json_file,"w") as f:
    json.dump(result_json,f)

1 /65 : left_Explorer_HD720_SN15385_04-00-39.png
Average model time usage:  2.8498799800872803
Average post time usage:  0.027257919311523438
2 /65 : left_Explorer_HD720_SN15385_05-00-63.png
Average model time usage:  1.6718285083770752
Average post time usage:  0.02410602569580078
3 /65 : left_Explorer_HD720_SN15385_05-00-52.png


KeyboardInterrupt: 